In [79]:
import pandas as pd
import numpy as np
import plotly.express as px
import random

In [80]:
poverty = pd.read_csv("cleaned_train.csv")

In [81]:
from sklearn.model_selection import train_test_split

# Assuming your DataFrame is named 'poverty_df'
# Shuffle the DataFrame
poverty_df_shuffled = poverty.sample(frac=1, random_state=42)

# Split the shuffled DataFrame into training (80%) and testing (20%) sets
poverty_train, poverty_test = train_test_split(poverty_df_shuffled, test_size=0.2, random_state=42)

In [82]:
poverty = poverty_train[['v2a1', 'v18q', 'r4m1', 'r4m3', 'r4t1', 'escolari', 'wall_condition', 'child_ratio', 'adult_ratio', 'education', 'SQBescolari', 'SQBhogar_nin', 'SQBovercrowding', 'Target']]
poverty_unseen = poverty_test[['v2a1', 'v18q', 'r4m1', 'r4m3', 'r4t1', 'escolari', 'wall_condition', 'child_ratio', 'adult_ratio', 'education', 'SQBescolari', 'SQBhogar_nin', 'SQBovercrowding', 'Target']]


In [83]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Encoding categorical variables
label_encoders = {}
for column in poverty.select_dtypes(include=['object']).columns:
    label_encoders[column] = LabelEncoder()
    poverty[column] = label_encoders[column].fit_transform(poverty[column])

# Splitting data into features and target variable
X = poverty.drop(columns=['Target'])
y = poverty['Target']

# Splitting data into train and test sets (80/20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define and train the neural network model
model = MLPClassifier(hidden_layer_sizes=(64, 32), activation='relu', solver='adam', max_iter=50)
model.fit(X_train, y_train)

# Predicting on test data
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy}')


Test Accuracy: 0.7129629629629629


In [84]:
from sklearn.feature_selection import SelectKBest, chi2

# Initialize SelectKBest
selector = SelectKBest(score_func=chi2, k=5)  # Adjust k as needed

# Fit selector to training data
selector.fit(X_train, y_train)

# Get the indices of the selected features
selected_indices = selector.get_support(indices=True)

# Get the names of the selected features
selected_features = X_train.columns[selected_indices]

# Print selected features
print("Selected Features:")
print(selected_features)


Selected Features:
Index(['v2a1', 'escolari', 'SQBescolari', 'SQBhogar_nin', 'SQBovercrowding'], dtype='object')


In [85]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import make_scorer, f1_score

X_train = poverty_train
y_train = X_train['Target']  # Assuming 'target_column' is the name of the target variable
X_train = X_train.drop('Target', axis=1)  # Remove the target column from the features

# Define the number of folds for cross-validation
k_folds = 5

# Define the cross-validation strategy
cv_strategy = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

# Define the Gradient Boosting classifier
gb_classifier = GradientBoostingClassifier()

# Perform k-fold cross-validation
cv_scores_gb = cross_val_score(gb_classifier, X_train, y_train, cv=cv_strategy, scoring='accuracy')
cv_scores_f1 = cross_val_score(gb_classifier, X_train, y_train, cv=cv_strategy, scoring=f1_scorer)

# Find the index of the model with the highest mean accuracy
best_model_index = cv_scores_gb.argmax()

# Retrieve the cross-validation scores of the best model
best_model_score = cv_scores_gb[best_model_index]

# Print the index, mean accuracy, and standard deviation of the best model
print(f"Best Model Index: {best_model_index}")
print(f"Best Model Mean Accuracy: {best_model_score:.4f}")
print(f"Best Model Standard Deviation: {cv_scores_gb.std():.4f}")

# You can also access the best model itself if needed
best_model = gb_classifier  # Assuming the best model is the Gradient Boosting Classifier used in cross-validation

# Train the best model on your full training data
best_model.fit(X_train,y_train)


Best Model Index: 0
Best Model Mean Accuracy: 0.8912
Best Model Standard Deviation: 0.0172


GradientBoostingClassifier()

In [86]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold

X_test = poverty_unseen
y_test = X_test['Target']  # Assuming 'target_column' is the name of the target variable
X_test = X_test.drop('Target', axis=1)  # Remove the target column from the features

# Define the number of folds for cross-validation
k_folds = 5

# Define the cross-validation strategy
cv_strategy = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

# Define the Gradient Boosting classifier
gb_classifier = GradientBoostingClassifier()

# Perform k-fold cross-validation
cv_scores_gb = cross_val_score(gb_classifier, X_test, y_test, cv=cv_strategy, scoring='accuracy')

# Find the index of the model with the highest mean accuracy
best_model_index = cv_scores_gb.argmax()

# Retrieve the cross-validation scores of the best model
best_model_score = cv_scores_gb[best_model_index]

# Print the index, mean accuracy, and standard deviation of the best model
print(f"Best Model Index: {best_model_index}")
print(f"Best Model Mean Accuracy: {best_model_score:.4f}")
print(f"Best Model Standard Deviation: {cv_scores_gb.std():.4f}")



Best Model Index: 1
Best Model Mean Accuracy: 0.7870
Best Model Standard Deviation: 0.0237


In [87]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Compute accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

# Compute precision
precision = precision_score(y_test, y_pred, average='weighted')
print(f'Precision: {precision:.4f}')

# Compute recall
recall = recall_score(y_test, y_pred, average='weighted')
print(f'Recall: {recall:.4f}')

# Compute F1-score
f1 = f1_score(y_test, y_pred, average='weighted')
print(f'F1-score: {f1:.4f}')


ValueError: Found input variables with inconsistent numbers of samples: [540, 432]